# Tutorial Walkthrough

## The SPM12w tutorial dataset

The Tutorial Dataset consists of three subjects with two runs of functional data and a defaced anatomy file for each participant. These data are shared with permission. The data consist of two runs of resting-state data, but for the purpooses of the tutorial we will treat this as functional data. Don't be suprised if the results are meaningless, there is no task to model. We're simply using this data to demonstrate how to run different models.

```{tip}
Whenever you are using spm12w, make sure Matlab is in the root directory of your study. In this cast that would be __spm12w\_tutorial\_data\_bids__.
```

## Step 1: Conversion from par/rec

The first step of running the tutorial data through its paces is to covnert that data from par/rec format to BIDS nifti files using __spm12w_prepare.m__

Although __spm12w_prepare.m__ has a number of options (see help), for the tutorial we will run it with only the _bidsgui_ argument which forces a gui to open so we can fix naming issues during the conversion to BIDS.

```octave
spm12w_prepare('bidsgui', true)
```

This will open up a window allowing you to select archive files located in the _./sourcedata/parrec_ directory. To continue, select all three *parrec\_tutorial0?\_900?DW\_13feb01\_s999?.zip* files and click __open__.

The next window will show two data series, these should be renamed to BIDS spec as in the image below. No other field needs to be altered This needs to be repeated for each of the three datasets.

```{figure} ../assets/ch4_prepare.png
---
width: 550px
name: ch4_prepare
---
spm12w_prepare: Type and modaily settings for the tutorial dataset.
```

## Step 2: Prepare data and generate tsvfiles

The next step for the tutorial dataset is to run two matlab files, the first will extract the anatomical MRI files and place them in the appropriate locations and the second will fix the json files generated during BIDS conversion for a small innacuracy in the TR. 

```{note}
The two steps below are only specific to the tutorial dataset. You will not have to do this on your own data. The tutorial dataset was collected a long time ago and the par/rec fields had some innacuracies in them. In addition, the need to manually copy to anatomy is due to the fact that these were defaced following conversion for anonimity and thus we cannot provide the raw par/rec files.
```

To run these scripts, type the following commands at the matlab prompt.

```octave
run('./sourcedata/code/anat_fixer.m')
run('./sourcedata/code/json_fixer.m')
```

The next step is to generate the tsv and json sidecars for the task (**task-tutorial_events.tsv**) and for the participants (**participants.tsv**). The tutorial data contains *.csv* files that contain all the necessary data to generate the task and participants *.tsv* files. 

```{tip}
You may want to inspect these files to understand how to generate task and participant files for your own studies.
```
To run these scripts, type the following commands at the matlab prompt.

```octave
spm12w_tsvmaker('sids',{'sub-tutorial01','sub-tutorial02','sub-tutorial03'}, ...
               'csvfile', './sourcedata/tsvfiles/task-tutorial_events_tsv.csv');
spm12w_tsvmaker('sids',{'sub-tutorial01','sub-tutorial02','sub-tutorial03'}, ...
               'csvfile', './sourcedata/tsvfiles/participants_tsv.csv');
```

The output should look similar to this:

```{figure} ../assets/ch4_tsvmaker.png
---
width: 550px
name: ch4_tsvmaker
---
spm12w_tsvmaker: Output after making tsv files for task and participants.
```

## Step 3: Validate the BIDS structure with the BIDS validator.

After preparing the data and generating the BIDS tsv files, you should use the [online BIDS validator tool](https://bids-standard.github.io/bids-validator/) to ensure the study conforms to the BIDS spec.

The BIDS validator expects that you select the directory containing the BIDS dataset without derivatives and sourcedata. In our case, that corresponds to the **rawdata** directory of the tutorial dataset. After selecting the **rawdata**, the output of the BIDS validator webpage should look similar to this:

```{figure} ../assets/ch4_validator.png
---
width: 550px
name: ch4_validate
---
BIDS validator: Output of the online BIDS validator tool.
```

You may see a warning that there are too few authors, this is because the BIDS spec doesn't believe it's possible for a dataset to have only one author!

## Step 4: Preprocess the fMRI data

The next step is to perform fMRI preprocess on the tutorial dataset. For this we will use the spm12w GUI.

Begin by typing `spm12w` in the Matlab command window. You should see the SPM12w GUI appear. 

```{figure} ../assets/ch_spm12w_gui.png
---
width: 550px
name: ch4_gui
---
SPM12w GUI: The SPM12w GUI allows easy access to most spm12w functions.
```

Next, we're going to preprocess all three subjects but as preprocessing is the longest step of fMRI analysis, we will run it in parallel using the Matlab Parallel Computing Toolbox. 

Before pressing the Preprocess button, turn on parallel processing (i.e., **\[ON] Parallel**) and then select 3 CPU cores and click the set button. You should see the following on the command prompt:

```
06/14/2024 16:14:27 [DEBUG] Parallel processing is [ON]
06/14/2024 16:14:30 [DEBUG] Parallel processing set to 3 cores...
```

Next, click the preprocess button and select all three tutorial participants. Click **done** when finished.

```{figure} ../assets/ch4_prepselect.png
---
width: 550px
name: ch4_select
---
The first three tutorial subjects are selected.
```

Finally, you must provide a parameters file that specifies the preprocessing parameters (See Ch3_parameters) for more information. The tutorial dataset comes with two files:
1. **p_tutorial.m**: Preprocessing and normalizing to MNI using the EPI template.
2. **p_tutorial_spm12norm.m**: Preprocessing and normalizing to MNI using the Anatomical file.

For this walkthrough we will use the **p_tutorial.m** file. Select that and hit done. Preprocessing will begin.

```{tip}
SPM12w will write a lot of details to the Matlab command window. It is advisable to monitor these the first time you preprocess a study in order to find any mistakes. In particular, pay attention to the *Slice time acquisition timing* the *number of runs* and the *number of vols* to ensure these look as expected.
```

After preprocessing, the following log files and QA files are created:
* epinorm.log: A logfile of everything that was printed to the Matlab command window.
* sub-tutorial0?_epinorm.pdf: A PDF of all the figures and spm12w specific QA generated during preprocesing.
* general qa image files: Mean, Stdev and temporal SNR nifti files for each preprocessed run.

The location of these files is printed to the command window after processing is complete. See below for an example.

```{figure} ../assets/ch4_logfiles.png
---
width: 550px
name: ch4_logfiles
---
Example of spm12w log and QA file locations after preprocessing.
```

## Step 5: Estimate GLMs

Once the data is preprocessed, the next step is to compute GLMs for each participant. For this we will again use the spm12w GUI. 

```{note}
If you have a lot of GLMs to compute, it may be useful to enable the Parallel Computing Toolbox in the SPM12w GUI. However, GLMs tend to be quick to compute and so there's less harm in running them serially. 
```

Once again, begin by typing `spm12w` in the Matlab command window. You should see the SPM12w GUI appear. 
1. Click on the **Estimate GLM** button.
2. Select the three tutorial participants and click done.
3. Select the glm_tutorial.m file and click done.

After GLMs are estimated, the location of various logs and figures are printed to the Matlab Command Window along with a summary of the GLM design. This should be inspected for accuracy. See the example below:

```{figure} ../assets/ch4_glmlogfiles.png
---
width: 550px
name: ch4_glmlogfiles
---
Example of spm12w log and QA file locations after glm estimation.
```

## Step 6: Compute Contrasts

The next step after estimating GLMs is to compute contrats of parameter estimates from the GLM model. This process is the same as the last.

Begin by typing `spm12w` in the Matlab command window. You should see the SPM12w GUI appear. 
1. Click on the **Compute Contrasts** button.
2. Select the three tutorial participants and click done.
3. Select the glm_tutorial.m file and click done.

```{note}
Although estimating GLMs and computing contrasts are seperate steps, they are so intertwined that we combine GLM parameters and contrast parameters into the same file (i.e., glm_tutorial.m) and the SPM12w GUI also contains a button to *Estimate and Contrast* in a single step.
```

After contrasts are computed, single-subject contrasts can be viewed in the derivatives directory specified in the glm file (i.e., **spm-stats-glm_tutorial**)

```{tip}
As with the previous steps, it is important to monitor the Matlab Command Window when computing contrasts. SPM12w will display the full contrast vector that it generates from the contrast specification in the GLM parameters file. It is good to check this vector for errors.
```

## Step 7: Random Effects Analysis

The next step will perform a 2nd-level Random Effects Analysis (RFX) on user specified contrasts. The steps here are the same as in computing contrasts.

Begin by typing `spm12w` in the Matlab command window. You should see the SPM12w GUI appear. 
1. Click on the **Random Effects** button.
2. Select the three tutorial participants and click done.
3. Select the glm_tutorial.m file and click done.

This will perform a one-sample t-test or a one-way anova (depending on the specifed parameters) for the conditions specified in the **glm_tutorial.m** file.

```{note}
Although conducting random effects, estimating GLMs and computing contrasts are seperate steps, the random effects analysis has so few parameters that it doesn't warrant a seperate file and we specify the RFX conditions and directory inside the glm_tutorial.m file.
```

After RFX analysis a summary of the conditions the analysis was performed on, as well as the location of a *.mat* file saving the parameters of the analysis is printed to the Matlab Command Window. The results of the RFX analysis can be found in the derivatives directory (i.e., **spm-group-stats-rfx_tutorial**)

## Step 8: Region-of-Interest (ROI) Analysis

The final step in the analysis of the tutorial data is to perform an ROI analysis. ROI analyses in spm12w are flexbile and allow for spherical ROI specification as well as image files. In addition, a range of statistics (one and two sample t-tests, correlations, descriptives) etc. can be specified in the ROI file. 

As with the previous steps, performing an ROI analysis with the SPM12w GUI requires specifying the subject and the associated parameters file.

Begin by typing `spm12w` in the Matlab command window. You should see the SPM12w GUI appear. 
1. Click on the **ROI Analysis** button.
2. Select the three tutorial participants and click done.
3. Select the roi_tutorial.m file and click done.

```{tip}
The contrast files used in the analysis as well as informatiuon on the spherical ROI voxel counts of all generated ROIs are printed to the Matlab Command Window. Be sure to check these for accuracy.
```

The ROI analysis will outpuit a *.csv* and *.txt* file containing the full ROI results and the summary statistics for each ROI. In addition, a *.mat* file is written containing all the parameters that existed at runtime to be kept as a record of the analysis in case the original roi parameters file is edited or lost.